<a href="https://colab.research.google.com/github/rcsjunior1987/Artificial-Intelligence/blob/master/genetic_algorithm_for_traveling_salesman_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [989]:
import random
import numpy as np

In [990]:
"""
  Class location
  , which each location is represented by its lattitude and longtude
"""

class Location:

    def __init__(self, lat, longt):
      self.lat = lat;
      self.longt = longt;

    # Calculates the distance between two locations
    def calc_distance(self, location):
      distance = np.sqrt((self.lat - location.lat) ** 2 + (self.longt - location.longt ) ** 2)
      return distance

    def __repr__(self):
        return "(" + str(self.lat) + " , " + str(self.longt) + ")"

In [991]:
class FitnessFunction:

  def __init__(self, route):
    self.route = route
    self.distance = 0
    self.fitness_value = 0.0

  # Method to calculate the fitness
  def calc_fitness(self):

    # If the fitness value has not been calculated yet
    if (self.fitness_value == 0):
      self.fitness_value = (1 / float(self.calc_route_distance()))
    return self.fitness_value

  # Method to route distance
  def calc_route_distance(self):

      if self.distance == 0:

        route_distance = 0

        # for each city in the route
        for i in range(0, len(self.route)):

          # Gets the city in the positon i
          from_city = self.route[i]

          # It starts as none
          to_city = None

          # If the next city is not the last one in the route
          if i + 1 < len(self.route):

            # It is assigned as the next city
            to_city = self.route[i+1]
          else:
            # It assigns the first city in the route
            to_city = self.route[0]

          # Acumulates the distance from the cities
          route_distance += from_city.calc_distance(to_city) 

        # Assigns the route distance into distance
        self.distance = route_distance

      # Returns the route distance
      return self.distance

In [992]:
class GA:

  # Method to create the population
  def create_population(self, locations, population_size):

    # Declares population as None
    population = []

    # The population 
    for i in range(0, population_size):
      population.append(self.create_route(locations))

    # Returns population
    return population

  # Creates a route from a list of locations
  def create_route(self, locations):
    route = random.sample(locations, len(locations))
    return route

  def rank_fitness(self, population):

    fitness_results = {}

    # If the fitness value has not been calculated yet
    for i in range(0, len(population)):

      # Calculates the routes distances
      fitness_results[i] = FitnessFunction(population[i]).calc_fitness()

    # Order by Asc
    return sorted(fitness_results.items(), reverse = True)
  

In [993]:
locations = []

# Creates a 10 dummy locations
for i in range(0, 10):
  locations.append(Location(lat=int(random.random() * 100), longt=int(random.random() * 100)))

# Creates an initial random population
population = GA().create_population(locations, 100)

# Prints the lowest distance
print("Initial distance: " + str(1 / GA().rank_fitness(population)[0][1]))

Initial distance: 530.5178735201262
